# <font color=blue>**PROYECTO** - Modelos Predictivos de Regresión Lineal Uni-Variable Sencillos</font>

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [42]:
#●	Usando sclicing con NumPy separar los datos en 2 datasets
PreciosCasas = np.load('proyecto_training_data.npy')
DatosEntrenamiento = PreciosCasas[0:int(PreciosCasas.shape[0]*.8)+1]
DatosValidacionPrueba = PreciosCasas[int(PreciosCasas.shape[0]*.8)+1:]
#print(DatosEntrenamiento)
#print(DatosValidacionPrueba.shape)

[[2.0850e+05 7.0000e+00 8.5600e+02 8.0000e+00 2.0030e+03 6.5000e+01]
 [1.8150e+05 6.0000e+00 1.2620e+03 6.0000e+00 1.9760e+03 8.0000e+01]
 [2.2350e+05 7.0000e+00 9.2000e+02 6.0000e+00 2.0010e+03 6.8000e+01]
 ...
 [2.4535e+05 8.0000e+00 1.6940e+03 7.0000e+00 2.0080e+03 6.4000e+01]
 [1.7300e+05 6.0000e+00 9.5900e+02 7.0000e+00 2.0000e+03 5.8000e+01]
 [2.3500e+05 6.0000e+00 1.2360e+03 7.0000e+00 1.9350e+03 1.2000e+02]]
(291, 6)


## ●	Análisis exploratorio de datos

In [74]:
#○	Para cada variable en el dataset calcular((usando numpy o pandas):
#   ■	media
#   ■	valor máximo
#   ■	valor mínimo 
#   ■	rango(peak to peak, no el rango del tensor que por ser vector sabemos que es 1)
#   ■	desviación estándar 

for i in range(DatosEntrenamiento.shape[1]):
    VarMedia = np.mean(DatosEntrenamiento[i])
    VarMax = np.max(DatosEntrenamiento[i])
    VarMin = np.min(DatosEntrenamiento[i])
    VarPeak = np.ptp(DatosEntrenamiento[i])
    VarDesv = np.std(DatosEntrenamiento[i])
    
    print('VARIABLE '+ str(i+1))
    print('Media               ->' + str(VarMedia))
    print('Max                 ->' + str(VarMax))
    print('Min                 ->' + str(VarMin))
    print('Rango               ->' + str(VarPeak))
    print('Desviacion Estandar ->' + str(VarDesv))
    #info = np.array([VarMedia,VarMax,VarMin,VarPeak,VarDesv]
    #AnalisisExploratorio = np.append(AnalisisExploratorio, ),axis=0)

VARIABLE 1
Media               ->35239.833333333336
Max                 ->208500.0
Min                 ->7.0
Rango               ->208493.0
Desviacion Estandar ->77487.55164845655
VARIABLE 2
Media               ->30805.0
Max                 ->181500.0
Min                 ->6.0
Rango               ->181494.0
Desviacion Estandar ->67396.91578254898
VARIABLE 3
Media               ->37750.333333333336
Max                 ->223500.0
Min                 ->6.0
Rango               ->223494.0
Desviacion Estandar ->83072.83863106053
VARIABLE 4
Media               ->23825.0
Max                 ->140000.0
Min                 ->7.0
Rango               ->139993.0
Desviacion Estandar ->51959.616841671705
VARIABLE 5
Media               ->42207.666666666664
Max                 ->250000.0
Min                 ->8.0
Rango               ->249992.0
Desviacion Estandar ->92930.43587663959
VARIABLE 6
Media               ->24314.0
Max                 ->143000.0
Min                 ->5.0
Rango               ->1

In [ ]:
#○	Para cada variable en el dataset usar seaborn(función distplot https://seaborn.pydata.org/generated/seaborn.distplot.html)  
#para graficar  un histograma de la variable.

